# Chat with LLM-Recommender Hybrid
- Also see writeup here: https://eugeneyan.com/writing/semantic-ids/

In [25]:
CATEGORY = "Video_Games"

In [26]:
import logging
import re
import sys
from typing import List

import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

In [27]:
# Create logger with the same format as src/logger.py
logger = logging.getLogger("hf-chat")
logger.setLevel(logging.INFO)

# Clear existing handlers to prevent duplicates
logger.handlers.clear()

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter("%(asctime)s - %(message)s", datefmt="%H:%M:%S"))
logger.addHandler(handler)
logger.propagate = False  # Prevent duplicate messages

In [28]:
# Device selection
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
logger.info(f"Using device: {device}")

20:40:17 - Using device: cuda


## Load Model and Dataset from Hugging Face Hub

In [29]:
# Define repository names based on category
category_lower = CATEGORY.lower().replace("_", "-")

# Repository names
MAPPING_REPO = f"eugeneyan/{category_lower}-semantic-ids-mapping"
MODEL_REPO = f"eugeneyan/semantic-id-qwen3-8b-{category_lower}"

logger.info("Loading from repositories:")
logger.info(f"  Dataset: {MAPPING_REPO}")
logger.info(f"  Model: {MODEL_REPO}")

20:40:17 - Loading from repositories:
20:40:17 -   Dataset: eugeneyan/video-games-semantic-ids-mapping
20:40:17 -   Model: eugeneyan/semantic-id-qwen3-8b-video-games


In [30]:
# Load the dataset from Hugging Face Hub
dataset = load_dataset(MAPPING_REPO, split="train")
mapping_df = dataset.to_pandas()

logger.info(f"✓ Loaded {len(mapping_df):,} products with semantic IDs and titles")
logger.info(f"Columns: {list(mapping_df.columns)}")
mapping_df.head()

20:40:18 - ✓ Loaded 66,097 products with semantic IDs and titles
20:40:18 - Columns: ['parent_asin', 'semantic_id', 'title']


,parent_asin,semantic_id,title
0,B000FH0MHO,<|sid_start|><|sid_32|><|sid_486|><|sid_636|><...,Dash 8-300 Professional Add-On
1,B00Z9TLVK0,<|sid_start|><|sid_188|><|sid_400|><|sid_518|>...,NBA 2K17 - Early Tip Off Edition - PlayStation 4
2,B002WH4ZJG,<|sid_start|><|sid_198|><|sid_262|><|sid_684|>...,Thrustmaster Elite Fitness Pack for Nintendo Wii
3,B0001ZNU56,<|sid_start|><|sid_65|><|sid_483|><|sid_757|><...,"Spongebob Squarepants, Vol. 1"
4,B009C9E8JY,<|sid_start|><|sid_154|><|sid_414|><|sid_585|>...,Set of 4 Bullet Buttons Nickel+Brass for Plays...


In [31]:
# Load model and tokenizer from Hugging Face Hub
import os
import warnings

# Disable all progress bars and widgets to avoid compatibility issues
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '1'
os.environ['TQDM_DISABLE'] = '1'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
os.environ['DISABLE_TELEMETRY'] = '1'

# Suppress widget warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

logger.info(f"Loading model from {MODEL_REPO}...")
logger.info("This may take a few minutes for the initial download (~16GB)")

# Load model - ignore widget errors as they don't affect functionality
model = AutoModelForCausalLM.from_pretrained(
    MODEL_REPO, 
    dtype=torch.bfloat16, 
    device_map=device, 
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True,
    padding_side="left",  # Important for generation
)

# Ensure tokenizer has pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

logger.info("✓ Model loaded successfully")
logger.info(f"Model device: {next(model.parameters()).device}")

20:40:19 - Loading model from eugeneyan/semantic-id-qwen3-8b-video-games...
20:40:19 - This may take a few minutes for the initial download (~16GB)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

20:40:26 - ✓ Model loaded successfully
20:40:26 - Model device: cuda:0


## Helper Functions

In [32]:
def parse_semantic_id(semantic_id: str) -> List[str]:
    """
    Parse a semantic ID string into its component levels.

    Example input: '<|sid_start|><|sid_127|><|sid_45|><|sid_89|><|sid_12|><|sid_end|>'
    Returns: ['<|sid_127|>', '<|sid_45|>', '<|sid_89|>', '<|sid_12|>']
    """
    # Remove start and end tokens
    sid = semantic_id.replace("<|sid_start|>", "").replace("<|sid_end|>", "")

    # Extract all sid tokens
    pattern = r"<\|sid_\d+\|>"
    levels = re.findall(pattern, sid)

    return levels


def map_semantic_id_to_titles(semantic_id_str: str, mapping_df: pd.DataFrame) -> dict:
    """
    Map a semantic ID to titles with 4-token exact match and 3-token fallback.

    Returns:
        dict with 'match_level', 'titles', and 'count' keys
    """
    # Parse the input semantic ID
    levels = parse_semantic_id(semantic_id_str)

    if not levels:
        return {"match_level": 0, "titles": [], "count": 0}

    # First try exact match (all 4 tokens)
    exact_matches = mapping_df[mapping_df["semantic_id"] == semantic_id_str]
    if len(exact_matches) > 0:
        titles = exact_matches["title"].tolist()
        return {"match_level": 4, "titles": titles, "count": len(titles), "match_type": "exact"}

    # Fallback to prefix matching (3 tokens, then 2, then 1)
    for depth in range(min(3, len(levels)), 0, -1):
        # Build the prefix for this depth
        prefix = "<|sid_start|>" + "".join(levels[:depth])

        # Find matches
        matches = mapping_df[mapping_df["semantic_id"].str.startswith(prefix)]

        if len(matches) > 0:
            # Found matches at this level
            titles = matches["title"].tolist()
            return {
                "match_level": depth,
                "titles": titles[:5],  # Limit to 5 for display
                "count": len(titles),
                "match_type": "prefix",
                "prefix_used": prefix,
            }

    # No matches found at any level
    return {"match_level": 0, "titles": [], "count": 0, "match_type": "none"}

In [33]:
def extract_semantic_ids_from_text(text: str) -> List[str]:
    """
    Extract all semantic IDs from a text string.

    Returns list of full semantic IDs found in the text.
    """
    # Pattern to match complete semantic IDs
    pattern = r"<\|sid_start\|>(?:<\|sid_\d+\|>)+<\|sid_end\|>"
    semantic_ids = re.findall(pattern, text)
    return semantic_ids


def replace_semantic_ids_with_titles(text: str, mapping_df: pd.DataFrame = None, show_match_level: bool = True) -> str:
    """
    Replace all semantic IDs in text with their corresponding titles.

    Args:
        text: Input text containing semantic IDs
        mapping_df: DataFrame with semantic_id to title mapping
        show_match_level: Whether to append match info after the title

    Returns:
        Text with semantic IDs replaced by titles
    """
    if mapping_df is None:
        mapping_df = globals().get("mapping_df")
        if mapping_df is None:
            raise ValueError("No mapping_df provided or found in global scope")

    # Find all semantic IDs in the text
    semantic_ids = extract_semantic_ids_from_text(text)

    # Create a copy of the text to modify
    result_text = text

    # Replace each semantic ID with its title(s)
    for sid in semantic_ids:
        # Get matching titles
        match_result = map_semantic_id_to_titles(sid, mapping_df)

        if match_result["count"] > 0:
            # Use the first title if multiple matches
            title = match_result["titles"][0]

            # Add match level if requested
            if show_match_level:
                if match_result["match_type"] == "exact":
                    replacement = f'"{title}"'
                else:
                    replacement = f'"{title}" (L{match_result["match_level"]} match)'
            else:
                replacement = f'"{title}"'

            # If multiple matches, indicate this
            if match_result["count"] > 1:
                replacement += f" [+{match_result['count'] - 1} similar]"
        else:
            # No match found
            replacement = "[Unknown Item]"

        # Replace the semantic ID with the title
        result_text = result_text.replace(sid, replacement)

    return result_text

## How a Semantic ID looks like

In [34]:
item = mapping_df.sample(1)
title = item['title'].item()
logger.info(f"Item Name: {title if len(title) < 80 else title[:80] + '...'}")
logger.info(f"Regular ID: {item['parent_asin'].item()}")
semantic_id_tokens = tokenizer.encode(item['semantic_id'].item())
logger.info(f"Semantic ID: {item['semantic_id'].item()}")
logger.info(f"Semantic ID as tokens: {semantic_id_tokens}")

20:40:26 - Item Name: Supreme Ruler - Cold War [Download]
20:40:26 - Regular ID: B005CWFD5I
20:40:26 - Semantic ID: <|sid_start|><|sid_21|><|sid_478|><|sid_738|><|sid_768|><|sid_end|>
20:40:26 - Semantic ID as tokens: [151670, 151693, 152150, 152410, 152440, 151671]


## Chat with Model

In [35]:
GLOBAL_MESSAGES = []
SYSTEM_PROMPT = """
Qwen, you are a helpful AI assistant that understands and works with semantic IDs for product recommendations.

Semantic IDs are hierarchical identifiers in the format <|sid_start|><|sid_0|><|sid_256|><|sid_512|><|sid_768|><|sid_end|> that encode product information and relationships.
""".strip()

In [36]:
def clean_output(text: str) -> str:
    """Remove known special tokens from output."""
    special_tokens = ["<think>", "</think>", "<|im_end|>", "<|im_start|>", "<|endoftext|>"]
    for token in special_tokens:
        text = text.replace(token, "")
    return text.strip()


def chat(
    text_input: str,
    new_convo: bool = True,
    temperature: float = 0.3,
    max_new_tokens: int = 512,
    stream: bool = True,
    mapping_df: pd.DataFrame = None,
) -> str:
    """
    Chat with the model, maintaining conversation history in GLOBAL_MESSAGES.

    Args:
        text_input: User input text
        new_convo: If True, clear global message history for a fresh start
        temperature: Generation temperature
        max_new_tokens: Maximum tokens to generate
        stream: Whether to stream output
        mapping_df: DataFrame for semantic ID mapping

    Returns:
        Generated response text
    """
    global GLOBAL_MESSAGES

    if mapping_df is None:
        mapping_df = globals().get("mapping_df")

    # Handle conversation history
    if new_convo:
        GLOBAL_MESSAGES = []
    else:
        # Display previous conversation turns if continuing
        if GLOBAL_MESSAGES:
            logger.info(f"{'=' * 41} Conversation History {'=' * 41}")
            for i, msg in enumerate(GLOBAL_MESSAGES, 1):
                role = msg["role"].upper()
                content = replace_semantic_ids_with_titles(msg["content"], mapping_df)
                logger.info(f"[Turn {(i+1)//2}] {role}: {content}")
            logger.info(f"{'=' * 35} Current Turn {'=' * 35}")

    GLOBAL_MESSAGES.append({"role": "user", "content": text_input})

    # Build messages with system prompt
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + GLOBAL_MESSAGES

    # Log input with readable semantic IDs
    readable_input = replace_semantic_ids_with_titles(text_input, mapping_df)
    logger.info(f"USER: {readable_input}")
    logger.info(f"{'=' * 20} START RAW MODEL OUTPUT (WITH SEMANTIC IDS) {'=' * 20}")

    # Prepare input for model
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    # Generate response
    streamer = TextStreamer(tokenizer, skip_prompt=True) if stream else None

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.7,
            top_k=20,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            streamer=streamer,
        )

    # Decode only the newly generated tokens
    input_length = inputs["input_ids"].shape[1]
    generated_tokens = output[:, input_length:]
    generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

    # Clean output
    generated_text = clean_output(generated_text)

    # Add to conversation history
    GLOBAL_MESSAGES.append({"role": "assistant", "content": generated_text})

    # Log output with readable semantic IDs
    logger.info(f"{'=' * 21} END RAW MODEL OUTPUT (WITH SEMANTIC IDS) {'=' * 21}")
    readable_output = replace_semantic_ids_with_titles(generated_text, mapping_df)
    logger.info(f"ASSISTANT: {readable_output}")

    return generated_text

## Single-Turn Conversations

In [37]:
# Provide input of user past interactions and get recommendation
INPUT = """User: <|sid_start|><|sid_64|><|sid_313|><|sid_637|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_447|><|sid_706|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_355|><|sid_734|><|sid_768|><|sid_end|>\n<|rec|>""".strip()
response = chat(INPUT)

20:40:27 - USER: User: "Halo 3 Limited Edition -Xbox 360", "Lost Planet: Extreme Condition - Xbox 360", "Lost Planet: Extreme Condition Collector's Edition (Amazon.com Exclusive Cell Included) -Xbox 360"
<|rec|>
20:40:27 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================


<|sid_start|><|sid_64|><|sid_351|><|sid_733|><|sid_768|><|sid_end|><|im_end|>
20:40:27 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:27 - ASSISTANT: "Mass Effect - Xbox 360"


In [38]:
# Provide input of single past item and get similar item
INPUT = """Customers who bought <|sid_start|><|sid_201|><|sid_311|><|sid_758|><|sid_768|><|sid_end|> also bought:\n<|rec|>""".strip()
response = chat(INPUT)

20:40:28 - USER: Customers who bought "New Super Mario Bros. 2" also bought:
<|rec|>
20:40:28 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|><|im_end|>
20:40:28 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:28 - ASSISTANT: "The Legend of Zelda: Ocarina of Time 3D"


In [39]:
# Steering recommendations given an item and attribute (Xbox)
INPUT = """Recommend Xbox games similar to <|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|>:\n<|rec|>""".strip()
response = chat(INPUT)

20:40:28 - USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D":
<|rec|>
20:40:28 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_167|><|sid_306|><|sid_646|><|sid_768|><|sid_end|>, <|sid_start|><|sid_167|><|sid_271|><|sid_578|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_271|><|sid_576|><|sid_768|><|sid_end|><|im_end|>
20:40:29 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:29 - ASSISTANT: "Halo: Combat Evolved Anniversary", "Halo Reach - Legendary Edition", "Fallout: New Vegas - Xbox 360 Ultimate Edition"


In [40]:
# Steering recommendations given an item and attribute (Playstation)
INPUT = """Recommend PS4 games similar to <|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|>:\n<|rec|>""".strip()
response = chat(INPUT)

20:40:30 - USER: Recommend PS4 games similar to "The Legend of Zelda: Ocarina of Time 3D":
<|rec|>
20:40:30 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_139|><|sid_356|><|sid_594|><|sid_768|><|sid_end|>, <|sid_start|><|sid_139|><|sid_382|><|sid_741|><|sid_769|><|sid_end|>, <|sid_start|><|sid_139|><|sid_476|><|sid_560|><|sid_768|><|sid_end|><|im_end|>
20:40:31 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:31 - ASSISTANT: "Final Fantasy XV Deluxe Edition - PlayStation 4", "Uncharted 4: A Thief's End - PlayStation 4", "Killzone: Shadow Fall (PlayStation 4)"


In [41]:
# Provide natural language chat input and get item recommendations
INPUT = """I like scifi and action games.\n<|rec|>""".strip()
response = chat(INPUT)

20:40:31 - USER: I like scifi and action games.
<|rec|>
20:40:31 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_8|><|sid_317|><|sid_736|><|sid_768|><|sid_end|>, <|sid_start|><|sid_64|><|sid_428|><|sid_721|><|sid_768|><|sid_end|>, <|sid_start|><|sid_219|><|sid_411|><|sid_753|><|sid_768|><|sid_end|><|im_end|>
20:40:32 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:32 - ASSISTANT: "Vanquish - Playstation 3", "Singularity - Xbox 360", "Lost Planet 3 - Playstation 3"


In [42]:
# Provide natural language chat input and get item recommendations
INPUT = """I like animal and cute games.\n<|rec|>""".strip()
response = chat(INPUT)

20:40:32 - USER: I like animal and cute games.
<|rec|>
20:40:32 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_173|><|sid_324|><|sid_764|><|sid_768|><|sid_end|>, <|sid_start|><|sid_201|><|sid_397|><|sid_738|><|sid_769|><|sid_end|>, <|sid_start|><|sid_173|><|sid_305|><|sid_587|><|sid_768|><|sid_end|><|im_end|>
20:40:33 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:33 - ASSISTANT: "Animal Crossing: New Leaf", "Disney Magical World - Nintendo 3DS", "Nintendogs + Cats: Toy Poodle and New Friends"


In [43]:
# Provide item to get recommendation and explanation
INPUT = """I just finished <|sid_start|><|sid_125|><|sid_417|><|sid_656|><|sid_768|><|sid_end|>. Suggest another <|rec|> and explain why:""".strip()
response = chat(INPUT)

20:40:34 - USER: I just finished "Dragon Quest Heroes II Explorer's Edition - PlayStation 4". Suggest another <|rec|> and explain why:
20:40:34 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_139|><|sid_289|><|sid_534|><|sid_768|><|sid_end|>

If you liked Dragon Quest Heroes II, you might like Nights of Azure because both are action RPGs on PlayStation 4 with a focus on combat and character progression. Both games offer a fantasy setting with a strong narrative, appealing to players who enjoy immersive single-player experiences with engaging stories and varied combat mechanics.<|im_end|>
20:40:36 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:36 - ASSISTANT: "Nights of Azure - PlayStation 4"

If you liked Dragon Quest Heroes II, you might like Nights of Azure because both are action RPGs on PlayStation 4 with a focus on combat and character progression. Both games 

In [44]:
# Provide list of items and get suggested bundle title and description
INPUT = """Suggest a name and description for a bundle of <|sid_start|><|sid_8|><|sid_454|><|sid_630|><|sid_768|><|sid_end|>, <|sid_start|><|sid_219|><|sid_453|><|sid_620|><|sid_768|><|sid_end|>, <|sid_start|><|sid_219|><|sid_469|><|sid_585|><|sid_768|><|sid_end|>""".strip()
response = chat(INPUT)

20:40:37 - USER: Suggest a name and description for a bundle of "Resistance 2 Collector's Edition - Playstation 3", "PS3 Killzone Trilogy Collection - 2 Disc", "The Last of Us - PlayStation 3"
20:40:37 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

PS3 Action & Survival Shooter Essentials: Experience intense first-person and third-person shooter action with this bundle of critically acclaimed PS3 titles, perfect for fans of survival and combat.<|im_end|>
20:40:38 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:38 - ASSISTANT: PS3 Action & Survival Shooter Essentials: Experience intense first-person and third-person shooter action with this bundle of critically acclaimed PS3 titles, perfect for fans of survival and combat.


## Multi-Turn Conversations

In [45]:
# 1st turn: Ask for games similar to Mario Kart
INPUT = "I'm looking for games similar to Mario Kart. <|rec|>"
response1 = chat(INPUT)

20:40:38 - USER: I'm looking for games similar to Mario Kart. <|rec|>
20:40:38 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_131|><|sid_492|><|sid_639|><|sid_768|><|sid_end|>, <|sid_start|><|sid_145|><|sid_480|><|sid_617|><|sid_768|><|sid_end|>, <|sid_start|><|sid_145|><|sid_290|><|sid_620|><|sid_768|><|sid_end|><|im_end|>
20:40:39 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:39 - ASSISTANT: "CTR: Crash Team Racing", "Crazy Taxi 2 - Sega Dreamcast", "Mario Kart: Super Circuit"


In [46]:
# 2nd turn: Tweak it for Xbox
INPUT = "How about something similar but for Xbox? <|rec|>"
response2 = chat(INPUT, new_convo=False)

20:40:40 - ========================================= Conversation History =========================================
20:40:40 - [Turn 1] USER: I'm looking for games similar to Mario Kart. <|rec|>
20:40:40 - [Turn 1] ASSISTANT: "CTR: Crash Team Racing", "Crazy Taxi 2 - Sega Dreamcast", "Mario Kart: Super Circuit"
20:40:40 - =================================== Current Turn ===================================
20:40:40 - USER: How about something similar but for Xbox? <|rec|>
20:40:40 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_167|><|sid_503|><|sid_522|><|sid_768|><|sid_end|>, <|sid_start|><|sid_167|><|sid_503|><|sid_522|><|sid_768|><|sid_end|>, <|sid_start|><|sid_183|><|sid_420|><|sid_618|><|sid_768|><|sid_end|><|im_end|>
20:40:41 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:41 - ASSISTANT: "Grand Theft Auto: San Andreas - Xbox 360 Digital Code", "Grand Theft Au

In [47]:
# 3rd turn: Ask for bundle name
INPUT = "Suggest a name and description for the bundle"
response3 = chat(INPUT, new_convo=False)

20:40:41 - ========================================= Conversation History =========================================
20:40:41 - [Turn 1] USER: I'm looking for games similar to Mario Kart. <|rec|>
20:40:41 - [Turn 1] ASSISTANT: "CTR: Crash Team Racing", "Crazy Taxi 2 - Sega Dreamcast", "Mario Kart: Super Circuit"
20:40:41 - [Turn 2] USER: How about something similar but for Xbox? <|rec|>
20:40:41 - [Turn 2] ASSISTANT: "Grand Theft Auto: San Andreas - Xbox 360 Digital Code", "Grand Theft Auto: San Andreas - Xbox 360 Digital Code", "Tony Hawk's Underground - Xbox"
20:40:41 - =================================== Current Turn ===================================
20:40:41 - USER: Suggest a name and description for the bundle
20:40:41 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

Retro Racing & Open World Adventure Pack: Relive classic gaming with iconic racing titles and dive into expansive open-world adventures for a diverse retro gam

## More multi-turn conversation

In [48]:
# 1st turn: Xbox games similar to Zelda
INPUT = """Recommend Xbox games similar to <|sid_start|><|sid_201|><|sid_396|><|sid_608|><|sid_769|><|sid_end|>\n<|rec|>""".strip()
response = chat(INPUT)

20:40:43 - USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D"
<|rec|>
20:40:43 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_167|><|sid_360|><|sid_685|><|sid_768|><|sid_end|>, <|sid_start|><|sid_167|><|sid_360|><|sid_685|><|sid_768|><|sid_end|>, <|sid_start|><|sid_167|><|sid_360|><|sid_685|><|sid_768|><|sid_end|><|im_end|>
20:40:44 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) =====================
20:40:44 - ASSISTANT: "Pac-Man and the Ghostly Adventures - Xbox 360", "Pac-Man and the Ghostly Adventures - Xbox 360", "Pac-Man and the Ghostly Adventures - Xbox 360"


In [49]:
# 2nd turn: Making it multiplayer
INPUT = "What about something similar but for multiplayer on Wii?\n<|rec|>"
response = chat(INPUT, new_convo=False)

20:40:44 - ========================================= Conversation History =========================================
20:40:44 - [Turn 1] USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D"
<|rec|>
20:40:44 - [Turn 1] ASSISTANT: "Pac-Man and the Ghostly Adventures - Xbox 360", "Pac-Man and the Ghostly Adventures - Xbox 360", "Pac-Man and the Ghostly Adventures - Xbox 360"
20:40:44 - =================================== Current Turn ===================================
20:40:44 - USER: What about something similar but for multiplayer on Wii?
<|rec|>
20:40:44 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_91|><|sid_303|><|sid_658|><|sid_768|><|sid_end|>, <|sid_start|><|sid_91|><|sid_319|><|sid_660|><|sid_768|><|sid_end|>, <|sid_start|><|sid_91|><|sid_303|><|sid_675|><|sid_768|><|sid_end|><|im_end|>
20:40:45 - ===================== END RAW MODEL OUTPUT (WITH SEMANTIC IDS) ================

In [50]:
# 2nd turn: Making it multiplayer
INPUT = "Suggest similar games but for racing\n<|rec|>"
response = chat(INPUT, new_convo=False)

20:40:45 - ========================================= Conversation History =========================================
20:40:45 - [Turn 1] USER: Recommend Xbox games similar to "The Legend of Zelda: Ocarina of Time 3D"
<|rec|>
20:40:45 - [Turn 1] ASSISTANT: "Pac-Man and the Ghostly Adventures - Xbox 360", "Pac-Man and the Ghostly Adventures - Xbox 360", "Pac-Man and the Ghostly Adventures - Xbox 360"
20:40:45 - [Turn 2] USER: What about something similar but for multiplayer on Wii?
<|rec|>
20:40:45 - [Turn 2] ASSISTANT: "Mario & Sonic at the Sochi 2014 Olympic Winter Games - Nintendo Wii U", "Rayman Raving Rabbids TV Party", "Mario and Sonic at the Olympic Winter Games - Nintendo Wii"
20:40:45 - =================================== Current Turn ===================================
20:40:45 - USER: Suggest similar games but for racing
<|rec|>
20:40:45 - ==================== START RAW MODEL OUTPUT (WITH SEMANTIC IDS) ====================
<think>

</think>

<|sid_start|><|sid_183|><|sid_461|><